# Part I. ETL Pipeline for Pre-Processing the Files

This first part is used to pre-process the input data located inside the `event_data` folder.

#### Import Python packages 

In [4]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [5]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

C:\Users\gasta\Documents\git_repo\data_modeling_cassandra


#### Processing the files to create the data file csv that will be used for Apache Cassandra tables

In [6]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# print(len(full_data_rows_list))
# print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Quality check on the newly generated file.

In [7]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

To have Apache Cassandra run inside a Docker container, please follow these steps:

1. Make sure to have Docker installed locally.
2. Run `docker pull cassandra` to fetch latest image from DockerHub.
3. Run `docker run --name cassandra-container -p 9042:9042 -d cassandra:latest` to spin a container at port 9042.
4. To stop the container, run `docker stop cassandra-container`

If Apache Cassandra is installed elsewhere, the IP address inside the Cluster object must be change accordingly.

In [8]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [9]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifydb
                        WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [10]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Query 1:

`Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4`

Since the query is filtering by `sessionId` and `itemInSession`, these fields should be included in the `PRIMARY KEY`.



In [11]:
# set up song_info_by_session and select statement
create_query1_tbl = """CREATE TABLE IF NOT EXISTS song_info_by_session
                       (sessionId int, itemInSession int, artist text, song text, length text, PRIMARY KEY(sessionId, itemInSession));"""
try:
    session.execute(create_query1_tbl)
except Exception as e:
    print(e)
    
query1 = """SELECT artist, song, length FROM song_info_by_session 
            WHERE sessionId=338 AND itemInSession=4;"""                    

In [12]:
## populate song_info_by_session
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_info_by_session (sessionId, itemInSession, artist, song, length) VALUES "
        query = query + "(%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], line[5]))

In [13]:
## inspect song_info_by_session content
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    res = (row.artist, row.song, row.length)
    print("|".join(res))

Faithless|Music Matters (Mark Knight Dub)|495.3073


## Query2:

`Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182`

`sessionId` and `userId` are used for filtering, but `itemInSession` is used for ordering. The first two fileds have been groupped togheter, while the third is enforced as clustering column.

In [14]:
# set up song_info_by_user_session and select statement
create_query2_tbl = """CREATE TABLE IF NOT EXISTS song_info_by_user_session
                       (sessionId int, userId int, itemInSession int, artist text, song text, user text, PRIMARY KEY((sessionId, userId), itemInSession));"""
try:
    session.execute(create_query2_tbl)
except Exception as e:
    print(e)
    
query2 = """SELECT artist, song, user FROM song_info_by_user_session
            WHERE sessionId=182 AND userId=10;"""   

## populate song_info_by_user_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_info_by_user_session (sessionId, userId, itemInSession, artist, song, user) VALUES "
        query = query + "(%s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1] + ' ' + line[4]))

## inspect song_info_by_user_session content
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

for row in rows:
    res = (row.artist, row.song, row.user)
    print("|".join(res))

Down To The Bone|Keep On Keepin' On|Sylvie Cruz
Three Drives|Greece 2000|Sylvie Cruz
Sebastien Tellier|Kilometer|Sylvie Cruz
Lonnie Gordon|Catch You Baby (Steve Pitron & Max Sanna Radio Edit)|Sylvie Cruz


## Query 3:

`Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'`

`song` should be used to parti

In [15]:
# set up query3_tbl and select statement
create_query3_tbl = """CREATE TABLE IF NOT EXISTS query3_tbl
                       (user text, song text, PRIMARY KEY(song, user));"""
try:
    session.execute(create_query3_tbl)
except Exception as e:
    print(e)
    
query3 = """SELECT user FROM query3_tbl 
            WHERE song = 'All Hands Against His Own';"""   

## populate query3_tbl
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO query3_tbl (user, song) VALUES "
        query = query + "(%s, %s);"
        session.execute(query, (line[1] + ' ' + line[4], line[9]))

## inspect query3_tbl content
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print(row.user)


Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [16]:
try:
    session.execute("DROP TABLE song_info_by_session;")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE song_info_by_user_session;")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE query3_tbl;")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()